In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the root directory
root_dir = r'U:\NVL\HX\A0\results_experimental\Thermal Profile'

# Initialize lists for each data series
dts_x = []
dts_y = []
tc_x = []
tc_y = []
fb_x = []
fb_y = []

# Function to process each file
def process_file(file_path):
    try:
        xl = pd.ExcelFile(file_path)
        if 'SearchVoltage Results' in xl.sheet_names:
            df = pd.read_excel(file_path, sheet_name='SearchVoltage Results')
            required_cols = ['run_order', 'cmvprofiling.Max_DTS_Profile_avg', 
                           'cmvprofiling.Intec_TC_Profile_avg', 'cmvprofiling.Intec_FB_Profile_avg']
            if all(col in df.columns for col in required_cols):
                dts_x.extend(df['run_order'].dropna().tolist())
                dts_y.extend(df['cmvprofiling.Max_DTS_Profile_avg'].dropna().tolist())
                tc_x.extend(df['run_order'].dropna().tolist())
                tc_y.extend(df['cmvprofiling.Intec_TC_Profile_avg'].dropna().tolist())
                fb_x.extend(df['run_order'].dropna().tolist())
                fb_y.extend(df['cmvprofiling.Intec_FB_Profile_avg'].dropna().tolist())
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Traverse the directory structure
try:
    main_folders = [f for f in os.listdir(root_dir) if 'D3' in f or 'D4' in f]
    for main in main_folders:
        main_path = os.path.join(root_dir, main)
        for dirpath, dirnames, filenames in os.walk(main_path):
            for filename in filenames:
                if filename.endswith('.xlsx') and ('HotVmin' in filename or 'HotGNG' in filename):
                    file_path = os.path.join(dirpath, filename)
                    process_file(file_path)
except Exception as e:
    print(f"Error traversing directories: {e}")

# Create the plot
plt.figure(figsize=(12, 8))
plt.scatter(dts_x, dts_y, color='blue', label='cmvprofiling.Max_DTS_Profile_avg')
plt.scatter(tc_x, tc_y, color='green', label='cmvprofiling.Intec_TC_Profile_avg')
plt.scatter(fb_x, fb_y, color='red', label='cmvprofiling.Intec_FB_Profile_avg')

plt.title('Max_DTS_Profiling')
plt.xlabel('run_order')
plt.ylabel('Temperature')
plt.legend(loc='upper right')
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent label cutoff
plt.show()  # Commented out to avoid execution

: 